# Document Question Answering Chatbot

In [10]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate

In [11]:
import os
import textwrap
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

open.api_key = os.environ['OPENAI_API_KEY']

In [12]:
def ask_question_with_context(qa, question, chat_history):
    result = qa({"question": question, "chat_history": chat_history})
    print(f"\n{textwrap.fill(result['answer'])}\n")
    chat_history = [(question, result["answer"])]
    return chat_history

## Initialize ChromaDB

In [13]:
embeddings = OpenAIEmbeddings()
persist_directory = 'epiq_policies_chroma/'
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
print(vectordb._collection.count())

255


## Create the chain and a helper function

Initialize the chain we will use for question answering.

In [14]:
llm_model_name = "gpt-3.5-turbo"
#llm_model_name = "gpt-4"
llm = ChatOpenAI(model_name=llm_model_name, temperature=0)

retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k":5})

In [15]:
QUESTION_PROMPT = PromptTemplate.from_template("""Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.

    Chat History:
    {chat_history}
    Follow Up Input: {question}
    Standalone question:""")

In [16]:
qa = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    condense_question_prompt=QUESTION_PROMPT,
    return_source_documents=True,
    verbose=False)

In [17]:
chat_history = []

## Ask questions!

Now we can use the chain to ask questions!

In [ ]:
while True:
    query = input('you: ')
    if query == 'q':
        break
    chat_history = ask_question_with_context(qa, query, chat_history)

you:  What are the most significant elements of Epiq's data protection policy?



The most significant elements of Epiq's data protection policy
include:  1. Privacy by Design & Default: Epiq ensures that controls
for safeguarding, minimizing, and restricting access to personal data
are identified and included in the design of new information systems
or enhancements to existing systems.  2. Cross-Border Data Transfers:
Epiq has policies and procedures in place for identifying, responding
to, documenting, and closing incidents involving potential data
breaches. They also have a breach notification plan and procedures for
notifying associates, customers, and third parties in the event of a
breach of personal data.  3. Monitoring: The Epiq Data Protection
Director periodically reviews and identifies any new laws and
regulations or changes to existing laws and regulations that may
impact the requirements defined in the data protection policy.  4.
Training and Awareness: Epiq requires mandatory privacy awareness
training for all employees upon initial hire and annually 

you:  Tell me more about what the policy says about HIPAA.



Epiq's data protection policy states that they are responsible for
ensuring the security, confidentiality, integrity, and availability of
its information systems containing ePHI (electronic protected health
information) by implementing appropriate policies, procedures, and
controls. They will develop and maintain written policies and
procedures for the handling of ePHI, including its receipt, storage,
transmission, and disposal. Violations of the HIPAA Compliance Program
may result in disciplinary actions, including termination. Epiq
personnel are also required to report any violations or issues related
to HIPAA to their manager and the appropriate HIPAA Program Owner.



you:  Does the policy say more about ePHI?



Epiq's data protection policy states that they are responsible for
ensuring the security, confidentiality, integrity, and availability of
its information systems containing ePHI. They do this by implementing
appropriate policies, procedures, and controls to prevent, detect,
contain, and correct issues or concerns related to ePHI. They also
develop and maintain written policies and procedures for the routine
and non-routine handling of ePHI.

